This code (will be) set up to generate a forecasted impact map of a specified area.
The area in the example case is Houston, TX
The inundation extent is sourced from the National Water Model forecast. 
Depth model is calculated from the inundation extent and the national 3m DEM
The impact map is calculated from the depth and the CDC SVI scores at the census tract scale. 

In [ ]:
# edit code so that it runs automatically every hour
# use python.12.4 kernel
# THIS IS A WORK IN PROGRESS

Currently, the dependencies require numpy <2, so make sure your numpy package matches that requirement. If it does, skip the next two steps

In [1]:
import os
import numpy as np
import arcpy
from datetime import datetime

## Import packages and set the paths to the working directory and input files
TO DO: 
1. Replace the directory path with the path to the folder you want to work in
2. Replace dem_name with path to the dem you want to use (as a tif) 
3. Replace the inund_polygon with path to the area extent you want to map (as a .shp)

In [2]:
# Import packages
import os
import arcpy
from datetime import datetime

# Format the current datetime as a string in the desired format (e.g., YYYYMMDD_HHMMSS)
datetime = datetime.now().strftime("%Y%m%d_%H%M%S")


# Replace 'path/to/your/directory' with the path to your desired working directory
directory = 'F:/SI_2024/Forecast_test_5day/'


# Create a new folder within the current working directory
resultfolder = "5dayforecast"+datetime
os.makedirs(os.path.join(directory, resultfolder), exist_ok=True)
print(f"New folder created at: {os.path.join(directory, resultfolder)}")
infolder = directory+resultfolder+'/'


# set up input variables
dem_name = 'C:/Users/Lyn/Documents/Programming/Python_codes/fwdet/Forecast_test_5day/Houston_DEM.tif'
extent = 'C:/Users/Lyn/Documents/ArcGIS/CUAHSI_SI_2024/Houston_huc.shp'
svi = 'C:/Users/Lyn/Documents/Programming/Python_codes/Impact_Map_Forecasting/Texas_example/aligned_svi_raster.tif'

# Set the working directory
os.chdir(infolder)
# Verify the current working directory
print("Current Working Directory: ", os.getcwd())

# set up input and output variables
arcpy.env.overwriteOutput = True

# make a geodatabase within the infolder   
gdb_path = infolder 
# Create the geodatabase
arcpy.CreateFileGDB_management(out_folder_path = infolder, out_name='geodatabase.gdb')
#workspace
ws = arcpy.env.workspace = infolder

New folder created at: F:/SI_2024/Forecast_test_5day/5dayforecast20240708_222641
Current Working Directory:  F:\SI_2024\Forecast_test_5day\5dayforecast20240708_222641


In [3]:
%pip install geopandas rasterio shapely

## Query the NWM 5 day inundation exent. This section will download it as a shapefile, merge polygons, and prepare for depth calculaiton

In [4]:
# query the NOAA API for the most recent flood predictions
import requests
import geopandas as gpd

# Construct the query URL
# To change the query, you can go to https://maps.water.noaa.gov/server/rest/services/nwm
# Then select the link you want, copy it here, and add /0/query to the end
# e.g. https://link_to_service/0/query
query_url = "https://maps.water.noaa.gov/server/rest/services/nwm/mrf_gfs_5day_max_inundation_extent/MapServer/0/query"
params = {
    'where': '1=1',
    'outFields': '*',
    'outSR': '4326',  # Specify output spatial reference if needed
    'f': 'geojson',  # Request geojson output
    'returnGeometry': 'true'
}
# Make the request
response = requests.get(query_url, params=params)


# Check if the server is running correctly. A response of 500 means there is a server error. 
if response.status_code == 200:
    try:
        # Attempt to convert the response to a GeoDataFrame
        predictions = gpd.GeoDataFrame.from_features(response.json(), crs="EPSG:4326")
        print(predictions.head())
    except ValueError as e:
        # Handle JSON decoding errors
        print(f"Error decoding JSON: {e}")
else:
    # The request failed; print the status code and response text
    print(f"Request failed with status {response.status_code}: {response.text}")



# Convert to GeoDataFrame
predictions = gpd.GeoDataFrame.from_features(response.json(), crs="EPSG:4326")
print(predictions.head())


Request failed with status 500: <html lang="en">
<head>
<title>
Error: Error performing query operation</title>
<link href="/server/rest/static/main.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<table width="100%" class="userTable">
<tr>
<td class="titlecell">ArcGIS REST Framework</td>
</tr>
</table>
<table width="100%" class="navTable">
<tr valign="top">
<td class="breadcrumbs">
<a href="/server/rest/services">Home</a>
</td>
</tr>
</table>
<div class="cbody">
<br/>
<br/>
<b>Error: </b>Error performing query operation<br/>
<b>Code: </b>500<br/><br/>
<div style="color:#ff6666"> </div></div>
</body>
</html>



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

clip the predictions to the dem extent

In [8]:
import geopandas as gpd

# Specify the path to the shapefile
shapefile_path = "F:/SI_2024/5day_07182024_smooth.shp"

# Read the shapefile into a GeoDataFrame
predictions = gpd.read_file(shapefile_path)

# Print the GeoDataFrame
print(predictions.head())

   COUNT    SHAPE_Leng    SHAPE_Area  \
0  924.0  8.358015e+06  3.540893e+08   

                                            geometry  
0  MULTIPOLYGON (((-10666025.504 3471520.993, -10...  


In [6]:
# clip the predictions to the extent of interest - defined by the dem_name above

import geopandas as gpd
import rasterio
from shapely.geometry import box


# Open the DEM file with rasterio
with rasterio.open(dem_name) as dem:
    # Read the entire file into a numpy array
    bounds = dem.bounds


# Create a bounding box polygon from the DEM bounds
bounding_box_polygon = box(*bounds)

# Clip the predictions GeoDataFrame to the bounding box
clipped_predictions = gpd.clip(predictions, bounding_box_polygon)
predictions = clipped_predictions



In [9]:
# If predictions is empty, print a message and exit the script
import sys
if predictions.empty:
    print("No flood predictions in this area found.")
    sys.exit(0)

## FWDET2.1 calculates depth map from inundation extent
### Written by Dr.Sagy Cohen

In [10]:
# use kernel arcgispro-py3 (python 3.9.16)
import arcpy

In [11]:

#clip_dem = '' #[Optional] - If empty, the clip_dem will be calculated with the Clip_management function
#cost_raster ='' #[Optional] - If empty, the CostRaster will be calculated below
WaterDepthOutput = 'WaterDepth_i10_s0p5'
iterations = 10
SlopeFiltering = True
SlopeThreshold = 0.5
inund_polygon = predictions # This is a new addition!! is it correct??? is inund_polgon supposed to be the predictions?

In [12]:
# clean and create necessary variables
dem = arcpy.Raster(dem_name)
cell_size = dem.meanCellHeight
# Proper string representation of dem extent to be accepted by Clip_management method
extent = '{} {} {} {}'.format(dem.extent.XMin, dem.extent.YMin, dem.extent.XMax, dem.extent.YMax)

# Dont run these if cost_raster and clip_dem are already defined above
cost_raster = (((dem <= 0)*999)+1)
cost_raster.save('CostRaster.tif')

clip_dem = 'ClipDEM'

arcpy.management.Clip(dem, extent, clip_dem, inund_polygon, nodata_value= "-9999", clipping_geometry="ClippingGeometry", maintain_clipping_extent="NO_MAINTAIN_EXTENT")
clip_dem_ras = arcpy.Raster(clip_dem)

RuntimeError: Object: Error in executing tool

In [13]:
#  replacement of the above code with the following code

import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np



# Open the raster file
with rasterio.open(dem_name) as src:
    # Clip the raster with the polygon
    # The mask function returns an array of the clipped raster and the affine transform of the clipped raster
    out_image, out_transform = mask(src, inund_polygon.geometry, crop=True, nodata=-9999)
    
    # Copy the metadata of the source raster
    out_meta = src.meta.copy()
    
    # Update the metadata with new details (transform, height, width, nodata value)
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                     "nodata": -9999})

# Write the clipped raster to a new file
with rasterio.open('C:/Users/Lyn/Documents/Programming/Python_codes/Impact_Map_Forecasting/Input_files/demclip.tif', "w", **out_meta) as dest:
    dest.write(out_image)

ValueError: Input shapes do not overlap raster.

In [ ]:
# Calculate Boundary Raster
arcpy.env.overwriteOutput = True
polyline = infolder + '/polyline.shp'
arcpy.PolygonToLine_management(inund_polygon, polyline)
# Convert polyline to raster
with arcpy.EnvManager(snapRaster=clip_dem_ras):
    arcpy.conversion.PolylineToRaster(polyline, 'FID', 'linerast15', "MAXIMUM_LENGTH", "NONE", cell_size)
raster_polyline = arcpy.Raster('linerast15')
raster_polyline.save("rstr_poly")
# The input whose values will be used as the output cell values if the condition is false.
inFalseConstant = '#'
where_clause = 'VALUE >= 0'
#Extract the boundary cells elevation from DEM
boundary = arcpy.sa.Con(raster_polyline, dem, inFalseConstant, where_clause)
# boundary.save('boundary1')
#Smooth boundary raster
for i in range(iterations):
    OutRasTemp = arcpy.sa.FocalStatistics(boundary, "Rectangle 5 5 CELL", 'MEAN', 'DATA')
    boundary = arcpy.sa.Con(raster_polyline, OutRasTemp, inFalseConstant, where_clause)
    boundary.save('boundary'+str(i+1))
#Identify and remove ocean boundary cells
OutRasTemp = arcpy.sa.FocalStatistics(dem, 'Circle 2 CELL', 'MINIMUM', 'DATA') 
whereClause2 = 'VALUE > 0'
boundary = arcpy.sa.Con(OutRasTemp, boundary, inFalseConstant, whereClause2)
#boundary.save("boundaryAfterOcean")
if SlopeFiltering:
#calculate topo slope
    print('Calculating Slope')
    extent_clip = '{} {} {} {}'.format(boundary.extent.XMin, boundary.extent.YMin, boundary.extent.XMax, boundary.extent.YMax)
    with arcpy.EnvManager(extent=extent_clip):
        out_slope = arcpy.sa.Slope(dem, "PERCENT_RISE", 1, "GEODESIC", "METER")
        out_slope.save("Slope_m")
#Remove erroneous boundary cells 
    whereClause_slope = 'VALUE > ' + str(SlopeThreshold)
    boundary = arcpy.sa.Con(out_slope, boundary, inFalseConstant, whereClause_slope)
boundary.save("boundFinal")
print('Finished calculating the boundary raster')

In [ ]:
# Calculate water depth 

MULTIPLIER = 10000
boundary_int = arcpy.sa.Int(boundary * MULTIPLIER)
#boundary_int.save("boundary_int")
print('Running cost allocation')
with arcpy.EnvManager(snapRaster=None, extent="DEFAULT", mask=clip_dem):
    cost_alloc = arcpy.sa.CostAllocation(boundary_int, cost_raster, '#', '#', 'Value')

# Divide the result from the cost allocation function using the same constant used to create the integer
# representation of the boundary
cost_alloc = arcpy.sa.Float(cost_alloc) / MULTIPLIER
print('Cost Allocation raster generated')
print('Calculating estimated water depth')
water_depth = (cost_alloc - clip_dem_ras)
# Remove estimated water depths below 0 and change them to 0
water_depth = arcpy.sa.Con(water_depth > 0, water_depth,"#")

In [ ]:
# Run a low-pass filter
# ie smooth the water depth raster
print('Running low-pass Filter')
water_depth_filtered = arcpy.sa.Filter(water_depth, 'LOW', 'DATA')
waterDepthFilter2 = arcpy.sa.Con(clip_dem_ras, water_depth_filtered, '#', 'VALUE > 0')
#waterDepthFilter2.save(WaterDepthOutput+'_filtered')
print('Finished low-pass Filter calculation')
waterDepthFilter2

In [ ]:
# Export the waterdepth as a tif file
# Set the output file path
waterdepth_path = infolder + 'waterDepth.tif'

# Export the waterDepthFilter2 raster as a TIFF file
waterDepthFilter2.save(waterdepth_path)

# Print the output file path
print(f"Water depth filter exported as: {waterdepth_path}")

## Run SVI using depth map

#### Step 1: Load the ESRI Geodatabase and Clip the Data

In [ ]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors
import matplotlib.colorbar as colorbar
import rasterio
from rasterio.features import rasterize
from rasterio.plot import show
import folium
from rasterio.warp import calculate_default_transform, reproject, Resampling
from matplotlib.colors import Normalize
from rasterio.transform import from_bounds
from osgeo import gdal, ogr, osr
import os

# Caculate the Impact 

#### Step 1: read the svi raster and fim raster

In [ ]:
import rasterio

# Define the paths to the raster files - same as the output above
svi_raster_path = svi
fim_raster_path = waterdepth_path

def check_raster_properties(raster_path):
    with rasterio.open(raster_path) as src:
        print(f"Properties of {raster_path}:")
        print(f" CRS: {src.crs}")
        print(f" Cell size: {src.res}")
        print(f" Width: {src.width}, Height: {src.height}")
        print(f" Bounds: {src.bounds}")
        print()

# Read and check properties of SVI raster
check_raster_properties(svi_raster_path)

# Read and check properties of FIM raster
check_raster_properties(fim_raster_path)


#### Step 2: Reproject and Resample SVI Raster
Skip this if they have same projection and cell size

##### make sure that the svi map has the same crs and cell size of fim map

In [ ]:
import os
from osgeo import gdal
import rasterio

# Define the paths to the raster files
aligned_svi_raster_path = infolder + 'aligned_svi_raster2.tif'

# Open the FIM raster to get the projection, geotransform, and size
fim_ds = gdal.Open(fim_raster_path)
if fim_ds is None:
    raise RuntimeError(f"Failed to open FIM raster {fim_raster_path}")

fim_proj = fim_ds.GetProjection()
fim_geotransform = fim_ds.GetGeoTransform()
fim_width = fim_ds.RasterXSize
fim_height = fim_ds.RasterYSize

print(f"FIM Raster Properties:\n CRS: {fim_proj}\n Geotransform: {fim_geotransform}\n Width: {fim_width}, Height: {fim_height}")

# Reproject and resample the SVI raster to match the FIM raster
warp_options = gdal.WarpOptions(
    format='GTiff',
    outputBounds=(fim_geotransform[0], fim_geotransform[3] + fim_height * fim_geotransform[5], fim_geotransform[0] + fim_width * fim_geotransform[1], fim_geotransform[3]),
    width=fim_width,
    height=fim_height,
    dstSRS=fim_proj,
    xRes=fim_geotransform[1],
    yRes=-fim_geotransform[5],
    resampleAlg=gdal.GRA_Bilinear,
    targetAlignedPixels=True
)

print("Warp options set.")

aligned_svi_raster = gdal.Warp(aligned_svi_raster_path, svi_raster_path, options=warp_options)
if aligned_svi_raster is None:
    raise RuntimeError(f"gdal.Warp failed to create {aligned_svi_raster_path}")

print("SVI raster has been reprojected and resampled to match the FIM raster.")

# Verify the properties of the aligned SVI raster
def check_raster_properties(raster_path):
    with rasterio.open(raster_path) as src:
        print(f"Properties of {raster_path}:")
        print(f" CRS: {src.crs}")
        print(f" Cell size: {src.res}")
        print(f" Width: {src.width}, Height: {src.height}")
        print(f" Bounds: {src.bounds}")
        print()

# Check properties of the aligned SVI raster
check_raster_properties(aligned_svi_raster_path)


Reclassify fim

In [ ]:
import numpy as np
import rasterio

def reclassify_fim(input_path, output_path):
    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        nodata = src.nodata

        if nodata is not None:
            data = np.ma.masked_equal(data, nodata)
        
        # Reclassify the FIM data
        reclassified_data = np.digitize(data, bins=[0.01, 0.5, 1, 2], right=True)
        reclassified_data = np.where(data == 0, 0, reclassified_data)  # Class 0 for non-flooded areas

        # Preserve nodata values
        reclassified_data = np.ma.filled(reclassified_data, nodata)

        meta = src.meta
        meta.update(dtype='float32', nodata=-9999)

        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(reclassified_data.astype('float32'), 1)


reclassified_fim_path = infolder + 'aligned_fim.tif'

reclassify_fim(fim_raster_path, reclassified_fim_path)
print("Reclassified FIM raster has been saved.")


Create impact map

In [ ]:
def multiply_rasters(svi_path, fim_path, output_path):
    with rasterio.open(svi_path) as svi_src:
        svi_data = svi_src.read(1).astype('float32')
        svi_nodata = svi_src.nodata

    with rasterio.open(fim_path) as fim_src:
        fim_data = fim_src.read(1).astype('float32')
        fim_meta = fim_src.meta
        fim_nodata = fim_src.nodata

    if svi_data.shape != fim_data.shape:
        raise ValueError("The rasters do not have the same shape.")

    # Ensure that nodata values are handled correctly
    nodata_value = -9999
    svi_data = np.where(svi_data == svi_nodata, np.nan, svi_data)
    fim_data = np.where(fim_data == fim_nodata, np.nan, fim_data)

    # Multiply the rasters to get the impact map, ignoring NaN values
    impact_data = np.where(np.isnan(svi_data) | np.isnan(fim_data), nodata_value, svi_data * fim_data)

    # Save the impact map to a new raster file
    impact_meta = fim_meta.copy()
    impact_meta.update({
        'dtype': 'float32',
        'nodata': nodata_value
    })

    with rasterio.open(output_path, 'w', **impact_meta) as impact_dst:
        impact_dst.write(impact_data.astype('float32'), 1)

svi_raster_path = svi
impact_raster_path = infolder + 'impact_raster.tif'

multiply_rasters(svi_raster_path, reclassified_fim_path, impact_raster_path)
print("Impact map has been saved.")


Mask 0s in the impact map

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

def mask_zeros(input_path, output_path):
    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        nodata = src.nodata

        masked_data = np.where(data == 0, nodata, data)

        meta = src.meta
        meta.update(dtype='float32', nodata=nodata)

        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(masked_data.astype('float32'), 1)


masked_impact_raster_path = infolder + 'masked_impact_raster.tif'

mask_zeros(impact_raster_path, masked_impact_raster_path)
print("Masked impact map has been saved.")

def plot_raster(ax, raster_path, title):
    with rasterio.open(raster_path) as src:
        data = src.read(1)
        nodata = src.nodata
        if nodata is not None:
            data = np.ma.masked_equal(data, nodata)  # Mask NoData values

        img = ax.imshow(data, cmap='plasma', norm=Normalize(vmin=data.min(), vmax=data.max()))
        ax.set_title(title)
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        return img

# Create a figure with subplots
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the masked impact raster
img = plot_raster(ax, masked_impact_raster_path, "Masked Impact Raster")

# Add colorbars with size control
cbar = fig.colorbar(img, ax=ax, fraction=0.046, pad=0.04, shrink=0.4)
cbar.set_label('Impact Value')

# Adjust layout
plt.tight_layout()
plt.show()


Reclassify the impact map

In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

def classify_raster(input_path, output_path):
    with rasterio.open(input_path) as src:
        data = src.read(1).astype('float32')
        nodata = src.nodata

        # Mask nodata values
        data_masked = np.ma.masked_equal(data, nodata)
        
        # Identify the range of the data excluding nodata values
        min_value = data_masked.min()
        max_value = data_masked.max()
        print(f"Data range: {min_value} to {max_value}")

        # Classify the data into 4 equal intervals
        bins = np.linspace(min_value, max_value, 5)  # 5 bins will create 4 intervals
        classified_data = np.digitize(data_masked, bins, right=True)  # Digitize creates 1 to 4 classes

        # Convert the classified_data mask to nodata values
        classified_data[data_masked.mask] = nodata

        print(f"Classified data range: {np.min(classified_data)} to {np.max(classified_data)}")

        meta = src.meta
        meta.update(dtype='float32', nodata=nodata)

        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(classified_data.astype('float32'), 1)


classified_impact_raster_path = infolder + 'classified_impact_raster.tif'

classify_raster(masked_impact_raster_path, classified_impact_raster_path)
print("Classified impact map has been saved.")

def plot_raster(ax, raster_path, title):
    with rasterio.open(raster_path) as src:
        data = src.read(1)
        nodata = src.nodata
        if nodata is not None:
            data = np.ma.masked_equal(data, nodata)  # Mask NoData values

        img = ax.imshow(data, cmap='plasma', norm=Normalize(vmin=1, vmax=4))  # Set vmin and vmax to match class values
        ax.set_title(title)
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        return img

# Create a figure with subplots
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the classified impact raster
img = plot_raster(ax, classified_impact_raster_path, "Classified Impact Raster")

# Add colorbars with size control
cbar = fig.colorbar(img, ax=ax, fraction=0.046, pad=0.04, shrink=0.4)
cbar.set_label('Impact Class')

# Adjust layout
plt.tight_layout()
plt.show()


Make shapefile

In [ ]:
from osgeo import gdal, ogr
import numpy as np


def raster_to_shapefile(raster_path, shapefile_path):
    # Open the reclassified raster
    src_ds = gdal.Open(raster_path)
    if src_ds is None:
        raise RuntimeError(f"Failed to open raster {raster_path}")
    
    # Get the raster band
    src_band = src_ds.GetRasterBand(1)
    nodata = src_band.GetNoDataValue()
    
    # Create the output shapefile
    driver = ogr.GetDriverByName('ESRI Shapefile')
    if driver is None:
        raise RuntimeError("ESRI Shapefile driver not available.")
    
    dst_ds = driver.CreateDataSource(shapefile_path)
    if dst_ds is None:
        raise RuntimeError(f"Failed to create shapefile {shapefile_path}")
    
    srs = ogr.osr.SpatialReference()
    srs.ImportFromWkt(src_ds.GetProjection())
    
    dst_layer = dst_ds.CreateLayer('impact', srs=srs, geom_type=ogr.wkbPolygon)
    if dst_layer is None:
        raise RuntimeError(f"Failed to create layer in shapefile {shapefile_path}")
    
    field_defn = ogr.FieldDefn('FIV_Class', ogr.OFTInteger)
    dst_layer.CreateField(field_defn)
    
    # Polygonize the raster, skipping NoData values
    gdal.Polygonize(src_band, src_band.GetMaskBand(), dst_layer, 0, options=["8CONNECTED=8"], callback=None)
    
    # Remove features with the NoData value
    dst_layer.SetAttributeFilter(f"FIV_Class = {int(nodata)}")
    for feature in dst_layer:
        dst_layer.DeleteFeature(feature.GetFID())
    dst_layer.SetAttributeFilter(None)
    
    # Close the datasets
    src_ds = None
    dst_ds = None

# Path to the reclassified impact raster file
reclassified_raster_path = classified_impact_raster_path
shapefile_path = infolder + 'reclassified_impact_raster.shp'


# Convert the reclassified raster to a shapefile
raster_to_shapefile(reclassified_raster_path, shapefile_path)
print("Shapefile has been created.")


add column to attribute table of FIV map to describe the severity of the vulnerability

In [ ]:
from osgeo import ogr

def add_vulnerability_column(shapefile_path):
    # Open the shapefile
    driver = ogr.GetDriverByName('ESRI Shapefile')
    ds = driver.Open(shapefile_path, 1)  # Open in write mode
    if ds is None:
        raise RuntimeError(f"Failed to open shapefile {shapefile_path}")
    
    layer = ds.GetLayer()
    
    # Add new field for vulnerability levels
    vulnerability_field = ogr.FieldDefn('Vuln_Level', ogr.OFTString)
    vulnerability_field.SetWidth(32)
    layer.CreateField(vulnerability_field)
    
    # Define the vulnerability levels
    vulnerability_levels = {
        1: "Low Vulnerability",
        2: "Medium Vulnerability",
        3: "High Vulnerability",
        4: "Very High Vulnerability"    
    }
    
    # Update the new field based on FIV_Class
    for feature in layer:
        fiv_class = feature.GetField('FIV_Class')
        vulnerability_level = vulnerability_levels.get(fiv_class, "Unknown")
        feature.SetField('Vuln_Level', vulnerability_level)
        layer.SetFeature(feature)
    
    # Close the dataset
    ds = None
    print("Vulnerability level column has been added to the shapefile.")

# Path to the shapefile
shapefile_path = infolder+'ImpactMap.shp'

# Add the vulnerability level column
add_vulnerability_column(shapefile_path)


Convert to geojson and commit to github

In [ ]:
# Convert to GeoJSON

# Step 1: Load the shapefile
gdf = gpd.read_file(shapefile_path)

# Step 2: Convert to GeoJSON
geojson_str = gdf.to_json()

# Step 3: Save the GeoJSON to a file
output_path = infolder+'ImpactMap.geojson'  # Replace with your desired output file name
with open(output_path, 'w') as f:
    f.write(geojson_str)